<a href="https://colab.research.google.com/github/gyq1985/deepmindset/blob/main/ML%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install clearml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.10.1
    Uninstalling PyJWT-2.10.1:
      Successfully uninstalled PyJWT-2.10.1


In [2]:
!clearml-init

ClearML SDK setup process

Please create new clearml credentials through the settings page in your `clearml-server` web app (e.g. http://localhost:8080//settings/workspace-configuration) 
Or create a free account at https://app.clear.ml/settings/workspace-configuration

In settings page, press "Create new credentials", then press "Copy to clipboard".

Paste copied configuration here:
Traceback (most recent call last):
object address  : 0x7cf219a39000
object refcount : 2
object type     : 0x9d5ea0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
Exception ignored in atexit callback: <bound method finalize._exitfunc of <class 'weakref.finalize'>>
Traceback (most recent call last):
  File "/usr/lib/python3.11/weakref.py", line 666, in _exitfunc
^C


In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
from clearml import Task, Logger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from clearml import Task
from PIL import Image

# 🛠 手动设置 ClearML 服务器地址和访问密钥
Task.set_credentials(
    web_host='https://app.clear.ml',
    api_host='https://api.clear.ml',
    files_host='https://files.clear.ml',
    key='Q9JQUW7NG3L7UIGUQ99CH6APXN7CWX',
    secret='ZDyUBTNev2TSyADi6gkFMEPRwMUBughNu4uVKUPjH7UsImaBOTLh2B6nVU2CwvYQTcw'
)

# ✅ 然后正常写你的 Task.init
task = Task.init(project_name="VGG16", task_name="Pipeline Step 3 - Train Pneumonia Model")

# Step 1: Connect to ClearML
task = Task.init(project_name="VGG16", task_name="Pipeline Step 3 - Train Pneumonia Model")
logger = Logger.current_logger()

# Step 2: Get Args
args = {
    'dataset_task_id': '405caed14d034630b33cf083a9fcc28d',  # ⬅️ 记得替换成真实的 Step 2 任务ID
    'img_size': (224, 224),
    'batch_size': 32,
    'epochs': 20,
}
task.connect(args)
# # Remote execution
# task.execute_remotely()
# Step 3: Get Uploaded Dataset Paths
dataset_task = Task.get_task(task_id=args['dataset_task_id'])
train_dir = dataset_task.artifacts['train_dir'].get()
val_dir = dataset_task.artifacts['val_dir'].get()
class_indices = dataset_task.artifacts['class_indices'].get()

IMG_SIZE = args['img_size']
BATCH_SIZE = args['batch_size']

# Step 4: Setup ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Step 5: Build Lightweight CNN
model = models.Sequential([
    Input(shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_indices), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 6: Add Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_accuracy', save_best_only=True)

# Step 7: Train Model
history = model.fit(
    train_generator,
    epochs=args['epochs'],
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)
# Step 8: Upload best model as Artifact
task.upload_artifact(name="best_model", artifact_object="best_model.keras")

# Step 9: Plot training curves
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.savefig("training_curves.png")
img = Image.open("training_curves.png")
logger.report_image("training_curves", "Accuracy and Loss", iteration=0, image=img)
print("✅ Model training and logging done.")

ValueError: Task object can only be updated if created or in_progress [status=stopped fields=['hyperparams']]

In [ ]:
task.close()

KeyboardInterrupt: 

In [15]:
!pip install clearml-agent

In [16]:
!clearml-agent init

CLEARML-AGENT setup process
Configuration file already exists: /root/clearml.conf
Leaving setup. If you've previously initialized the ClearML SDK on this machine, manually add an 'agent' section to this file.


In [17]:
!pkill -f clearml-agent

In [18]:
# pipe.start(queue="vgg16")
!clearml-agent daemon --queue vgg16 --detached

Using environment access key CLEARML_API_ACCESS_KEY=Q9JQUW7NG3L7UIGUQ99CH6APXN7CWX
Using environment secret key CLEARML_API_SECRET_KEY=********
Current configuration (clearml_agent v1.9.3, location: /root/clearml.conf):
----------------------
sdk.storage.cache.default_base_dir = ~/.clearml/cache
sdk.storage.cache.size.min_free_bytes = 10GB
sdk.storage.direct_access.0.url = file://*
sdk.metrics.file_history_size = 100
sdk.metrics.matplotlib_untitled_history_size = 100
sdk.metrics.images.format = JPEG
sdk.metrics.images.quality = 87
sdk.metrics.images.subsampling = 0
sdk.metrics.tensorboard_single_series_per_graph = false
sdk.network.metrics.file_upload_threads = 4
sdk.network.metrics.file_upload_starvation_warning_sec = 120
sdk.network.iteration.max_retries_on_server_error = 5
sdk.network.iteration.retry_backoff_factor_sec = 10
sdk.network.file_upload_retries = 3
sdk.aws.s3.key = 
sdk.aws.s3.secret = ****
sdk.aws.s3.region = 
sdk.aws.s3.use_credentials_chain = false
sdk.aws.boto3.pool_c

Action failed <400/110: tasks.add_or_update_artifacts/v2.10 (Invalid task status: expected=created, status=stopped)> (task=85eabaadfd7441a3b94c5c841a16794e, artifacts=[{'key': 'best_model', 'type': 'custom', 'uri': 'https://files.clear.ml/VGG16/Pipeline%20Step%203%20-%20Train%20Pneumonia%20Model.85eabaadfd7441a3b94c5c841a16794e/artifacts/best_model/best_model.keras', 'content_size': 143603355, 'hash': '4b1f80a83f0ced8ecbea6d3796b17affa5728f1068b71096291113c0fb9112bb', 'timestamp': 1745924093, 'type_data': {'preview': 'best_model.keras - 143.6 MB\n'}}, {'key': 'notebook preview', 'type': 'custom', 'uri': 'https://files.clear.ml/VGG16/Pipeline%20Step%203%20-%20Train%20Pneumonia%20Model.85eabaadfd7441a3b94c5c841a16794e/artifacts/notebook%20preview/notebook_85eabaadfd7441a3b94c5c841a16794e.html', 'content_size': 334727, 'hash': '1b65b73704bb86c96fb6e77a5322fb5173a0acb0426b081cc150146a1bf54278', 'timestamp': 1745930331, 'type_data': {'preview': 'Click `FILE PATH` link', 'content_type': 'tex

KeyboardInterrupt: 